In [5]:
import json
from datasets import Dataset
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

# jsonl 파일 경로
file_path = "./data/INHA-DACON.jsonl"

# jsonl 파일 읽기
data = []
with open(file_path, "r") as f:
    for line in f:
        data.append(json.loads(line))

# 데이터셋 생성
dataset = Dataset.from_list(data)

# 컬럼 확인 (예시)
print(dataset.column_names)
# ['context', 'instruction', 'response']


['context', 'instruction', 'response']


In [6]:
model_name = "beomi/Llama-3-Open-Ko-8B"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# 전처리 함수 정의
def preprocess_function(examples):
    inputs = [f"{ctx} {inst}" for ctx, inst in zip(examples["context"], examples["instruction"])]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)
    
    # Response 토큰화
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["response"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_datasets = dataset.map(preprocess_function, batched=True)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Map:   0%|          | 0/33716 [00:00<?, ? examples/s]

/Users/sonhoyoung/Documents/Codes/Python/LLM/.venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [7]:
from peft import LoraConfig, get_peft_model

# LoRA 설정
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none"
)

model = AutoModelForCausalLM.from_pretrained(model_name)
model = get_peft_model(model, lora_config)


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

/Users/sonhoyoung/Documents/Codes/Python/LLM/.venv/lib/python3.10/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [8]:
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=8,
    warmup_steps=100,
    max_steps=200,
    learning_rate=3e-4,
    fp16=True,
    logging_steps=10,
    output_dir="./lora_llama"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets
)

trainer.train()


ValueError: fp16 mixed precision requires a GPU (not 'mps').

In [ ]:
model.save_pretrained("./lora_llama_trained")
tokenizer.save_pretrained("./lora_llama_trained")
